In [2]:
"""
Generating data for "Predicting Forest Regrowth in Amazonia given Land Use History"
Author: Ana Catarina Avila
Date: Dec 2023
"""

import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')


## Climate

### CWD

Calculate mean AET from MODIS and calculate CWD from the precipitation values as in Celso

Since MODIS only goes back to 2000, for now we are stuck with a fixed value for ET


In [4]:
# Function to calculate mean AET and add year property
# select for mature forests since the values can be put off by deforestation (causes lower ET)
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").filter(ee.Filter.eq("id", 18413)).geometry()

start = "2002-01-01"
end = "2020-12-01"

modis = ee.ImageCollection("MODIS/061/MOD16A2GF") \
         .filterDate(start, end) \
         .select('ET', 'ET_QC').map(lambda image: image.clip(roi).updateMask(mature_mask)
         .reproject(scale = 10000, crs = 'EPSG:4326').reduceResolution(ee.Reducer.median()))

months = ee.List.sequence(1, 12)
years = ee.List.sequence(1985, 2019)

NameError: name 'mature_mask' is not defined

In [114]:
# code sourced from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwise_extract(input, from_bit, to_bit):
   mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
   mask = ee.Number(1).leftShift(mask_size).subtract(1)
   return input.rightShift(from_bit).bitwiseAnd(mask)

def apply_QA_mask (image):
    QA = image.select('ET_QC')
    ET = image.select('ET').multiply(0.0125)  # multiply by the scale 0.1, divide by 8 to get daily info
    cloud_mask = bitwise_extract(QA, 3, 4).lte(0)
    qa_mask = bitwise_extract(QA, 5, 7).lte(1)
    mask = cloud_mask.And(qa_mask)
    return ET.updateMask(mask).set('system:time_start', image.get('system:time_start'))

# mask quality of pixels
modis_masked = modis.map(apply_QA_mask)

# Loop through the months and filter images
def et_monthly(month):
    eight_day_images = modis_masked.select('ET').filter(ee.Filter.calendarRange(month, month, 'month'))
    monthly_et = eight_day_images.median().multiply(30).reduceRegions(ecoregions, \
                                                                      reducer = ee.Reducer.median(), scale = 10000)
    return monthly_et.set('month', month)

monthly_et_list = ee.Image(months.map(et_monthly))

task = ee.batch.Export.image.toAsset(
    
)

### Terraclim and seasonality

Bring temperature and precipitation and calculate seasonality

In [1]:
terraclim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
              .filterDate('1985-01-01', '2019-12-31') \
              .map(lambda image : image.clip(roi) \
                  .reproject(scale = 10000, crs = 'EPSG:4326').reduceResolution(ee.Reducer.median()))

maxtemp = terraclim.select('tmmx').map(lambda image: image.multiply(0.1))
mintemp = terraclim.select('tmmn').map(lambda image: image.multiply(0.1))
radiation = terraclim.select('srad').map(lambda image: image.multiply(0.1))
prec = terraclim.select('pr')

NameError: name 'ee' is not defined

In [110]:
monthly_wd = prec.map(lambda image: image.subtract(100))
# monthly_wd_list = monthly_wd.toList(monthly_wd.size())



def cwd_monthly(current_image, previous_dict):
    current_image = ee.Image(current_image)
    previous_dict = ee.Dictionary(previous_dict)
    previous_image = ee.Image(previous_dict.get('result_image'))
    result_image = previous_image.add(current_image).where(previous_image.gt(0), 0) \
                                   .set('system:time_start', current_image.get('system:time_start'))
    return ee.Dictionary({
        'result_image': ee.Image(result_image),
        'cwd_list': ee.List(previous_dict.get('cwd_list')).add(result_image)})

# Use iterate to apply the function to each image
cwd_dict = ee.Dictionary(water_deficit.iterate(cwd_monthly, ee.Dictionary({'result_image': ee.Image(0),
                                                               'cwd_list': ee.List([])})))
cwd_list = ee.List(cwd_dict.get('cwd_list'))


def mcwd_yearly(year):
    # Calculate the start and end indices for the current year
    start = ee.Number(year).subtract(1985).multiply(12)
    end = start.add(12)
    # Slice the list to get the images for the current year
    yr = ee.List(cwd_list).slice(start, end)
    cwd_a = ee.ImageCollection.fromImages(yr)
    mcwd_a = cwd_a.reduce(ee.Reducer.min())
    return mcwd_a.set('year', year)

# Map the function over the range
annual_mcwd = ee.ImageCollection.fromImages(years.map(mcwd_yearly))

In [98]:
def seasonality_calc(y):
    yr_prec = prec.filter(ee.Filter.calendarRange(y, y, 'year'))
    # get mean monthly precipitation for the year
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month:month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    seasonality_index = sum_deviations.divide(total_annual_prec)
    return seasonality_index

yearly_SI = ee.ImageCollection.fromImages(years.map(seasonality_calc))

# Convert ImageCollection to single Image
yearly_SI_image = yearly_SI.toBands()
yearly_SI_image.bandNames()

In [102]:
def meantemp_calc(y):
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(y, y, 'year'))
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(y, y, 'year'))
    # Calculate the mean temperature directly
    mean_temp = yearly_maxtemp.reduce(ee.Reducer.mean()).add(yearly_mintemp.reduce(ee.Reducer.mean())).divide(2)
    return mean_temp

# Map the function over the years
mean_temp = ee.ImageCollection.fromImages(years.map(meantemp_calc))